In [1]:
project_name = 'rcnn-one_class_bottle_focus'

%mkdir /notebooks/Faster-RCNN/{project_name}
%cd /notebooks/Faster-RCNN/{project_name}

!python -m pip install pyyaml==5.1
import sys, os, distutils.core
import torch, torchvision

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch torchvision -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab
!pip install opencv-python

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

##RETREIVE LINK FROM ROBOFLOW#
!curl -L "https://app.roboflow.com/ds/1oy107zCMg?key=9UD5gUbkgV" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
#https://app.roboflow.com/ds/mlGUSDhnsp?key=bB5yusTtUD
###############################

from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train_{}".format(project_name), {}, "/notebooks/Faster-RCNN/{}/train/_annotations.coco.json".format(project_name), "/notebooks/Faster-RCNN/{}/train".format(project_name))
register_coco_instances("my_dataset_val_{}".format(project_name), {}, "/notebooks/Faster-RCNN/{}/valid/_annotations.coco.json".format(project_name), "/notebooks/Faster-RCNN/{}/valid".format(project_name))
register_coco_instances("my_dataset_test_{}".format(project_name), {}, "/notebooks/Faster-RCNN/{}/test/_annotations.coco.json".format(project_name), "/notebooks/Faster-RCNN/{}/test".format(project_name))

#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

/notebooks/Faster-RCNN/rcnn-one_class_bottle_focus
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-xowzc5ac
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-xowzc5ac
  Resolved https://github.com/facebookresearch/detectron2.git to commit 7d2e68dbe452fc422268d40ac185ea2609affca8
  Preparing metadata (setup.py) ... done
  Using cached pycocotools-2.0.7-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0
Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-bray9vzh
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-bray9vzh
  Resolved https://

In [2]:
#Reference: https://detectron2.readthedocs.io/en/latest/modules/config.html
#Reference: https://detectron2.readthedocs.io/en/latest/modules/solver.html

#from .detectron2.tools.train_net import Trainer
#from detectron2.engine import DefaultTrainer
# select from modelzoo here: https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines


from detectron2.config import get_cfg
#from detectron2.evaluation.coco_evaluation import COCOEvaluator
import os

cfg = get_cfg()
cfg.MODEL.RPN.IOU_THRESHOLDS = [0.2, 0.5]
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train_{}".format(project_name),)
cfg.DATASETS.TEST = ("my_dataset_val_{}".format(project_name),)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001


cfg.SOLVER.WARMUP_ITERS = 500
cfg.SOLVER.MAX_ITER = 5000 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (4000,4500)
cfg.SOLVER.GAMMA = 0.05

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 #your number of classes + 1

cfg.TEST.EVAL_PERIOD = 0


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[08/15 17:51:52 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, we

[08/15 17:51:53 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.9/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[08/15 17:52:03 d2.utils.events]:  eta: 0:30:33  iter: 19  total_loss: 2.392  loss_cls: 1.174  loss_box_reg: 0.6456  loss_rpn_cls: 0.4185  loss_rpn_loc: 0.06344    time: 0.3625  last_time: 0.3405  data_time: 0.0154  last_data_time: 0.0059   lr: 3.8962e-05  max_mem: 6864M
[08/15 17:52:13 d2.utils.events]:  eta: 0:30:34  iter: 39  total_loss: 1.815  loss_cls: 0.6845  loss_box_reg: 0.8412  loss_rpn_cls: 0.1682  loss_rpn_loc: 0.0713    time: 0.3792  last_time: 0.3782  data_time: 0.0066  last_data_time: 0.0066   lr: 7.8922e-05  max_mem: 6864M
[08/15 17:52:20 d2.utils.events]:  eta: 0:30:28  iter: 59  total_loss: 1.412  loss_cls: 0.452  loss_box_reg: 0.7525  loss_rpn_cls: 0.1087  loss_rpn_loc: 0.06298    time: 0.3745  last_time: 0.3338  data_time: 0.0065  last_data_time: 0.0059   lr: 0.00011888  max_mem: 6864M
[08/15 17:52:27 d2.utils.events]:  eta: 0:30:20  iter: 79  total_loss: 1.222  loss_cls: 0.3365  loss_box_reg: 0.8412  loss_rpn_cls: 0.04972  loss_rpn_loc: 0.05947    time: 0.3706  last

In [12]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test_{}".format(project_name), cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test_{}".format(project_name))
inference_on_dataset(trainer.model, val_loader, evaluator)

detectron2.evaluation.print_csv_format(results)

[08/15 18:52:14 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
[08/15 18:52:17 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
WARNING [08/15 18:52:17 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
WARNING [08/15 18:52:17 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[08/15 18:52:17 d2.data.datasets.coco]: Loaded 5078 images in COCO format from /notebooks/Faster-RCNN/rcnn-one_class_bottle_focus/test/_annotations.coco.json
[08/15 18:52:18 d2.data.build]: Distribution of instances among all 7 categories:
|   category    | #instances   |   category    | #instances   |  category  | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:----------:|:-------------|
| small-objec.. | 0            | cle

NameError: name 'results' is not defined